In [ ]:
import json
import os
import sys

import geopandas as gpd
import shapely.geometry as geom

# Re-use some of the libraries that are used by app
sys.path.append('../apps/arcade-pipeline')
sys.path.append('../apps/arcade-pipeline/processors')

from processors.xarray_utils import XarrayUtils
from processors.metadata_utils import MetadataUtils
from stac_catalog_processor import STACCatalogProcessor, EngineProcessRequest

## Set with your environment specific value

In [ ]:
%env ISENGARD_PRODUCTION_ACCOUNT=false
%env AWS_ACCESS_KEY_ID=
%env AWS_SECRET_ACCESS_KEY=
%env AWS_SESSION_TOKEN=
%env OUTPUT_BUCKET=
%env EVENT_BUS_NAME=
%env ARCADE_STAC_SERVER_URL=
%env AWS_BATCH_JOB_ID=
%env INPUT_PREFIX=
%env INPUT_FILENAME=

In [ ]:
engine_input = {"id": "01hwvwmm5d2crn4xx8k0s0a61z", "groupId": "01hwsqbq5trgzmd8k19mkhepz9", "name": "test_farm_area", "attributes": {}, "tags": {},
				"scheduleExpressionTimezone": "Australia/Perth", "scheduleExpression": "at(2024-04-01T11:37:00)", "createdBy": "willsia@amazon.com",
				"createdAt": "2024-05-02T05:32:36.910Z", "scheduleDateTime": "2024-05-02",
				"coordinates": [[-104.5079674, 39.9194752], [-104.4894065, 39.9193435], [-104.4893912, 39.9122295], [-104.5078877, 39.9123941], [-104.5079674, 39.9194752]],
				"polygonId": "01hwvwmv68evdbrm1z89ay269b", "regionId": "01hwvwmm5d2crn4xx8k0s0a61z", "resultId": "01hwyh9ehcbh3y8fjvms496d4y",
				"outputPrefix": "region=01hwvwmm5d2crn4xx8k0s0a61z/result=01hwyh9ehcbh3y8fjvms496d4y/output/polygon=01hwvwmv68evdbrm1z89ay269b"}

## Load Stac Datasets

In [ ]:
request = EngineProcessRequest.from_dict(engine_input)
processor = STACCatalogProcessor(request)
stac_assets, previous_ndvi_raster = processor.load_stac_datasets()

bucket_name = os.getenv("OUTPUT_BUCKET")
temp_dir = "{}/{}".format(os.getcwd(), "output")

print(stac_assets)

## Plot Region (Debugging)

In [ ]:
CRS = 'epsg:4326'
polygon = geom.Polygon(request.coordinates)
polygon_array = [polygon]
polygon_series = gpd.GeoSeries(polygon_array, crs=CRS)
aoi = gpd.GeoDataFrame(geometry=polygon_series, crs=CRS)
aoi.plot(figsize=(10, 8), alpha=0.5, edgecolor='k')
bounding_box = polygon_series.total_bounds

## 1. Calculate cloud removal bands

In [ ]:
# Perform cloud removal
stac_assets['scl_cloud_removed'] = XarrayUtils.remove_cloud(stac_assets[['scl']])
print(stac_assets)

### 1.a. Plot after cloud removal (debugging)

In [ ]:
from pystac.extensions.projection import ProjectionExtension
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pyproj import CRS

sentinel_epsg = ProjectionExtension.ext(processor.stac_item).epsg
output_crs = CRS.from_epsg(sentinel_epsg)

aoi_projected = aoi.to_crs(output_crs)

scl_color_def = [
	(0, '#000000', 'No data'),  # black
	(1, '#ff0000', 'Saturated or defective'),  # red
	(2, '#404040', 'Cast shadows'),  # gray
	(3, '#833c0b', 'Cloud shadows'),  # brown
	(4, '#04ff00', 'Vegetation'),  # green
	(5, '#ffff00', 'Not vegetated (bare soil)'),  # yellow
	(6, '#0000cc', 'Water'),  # blue
	(7, '#757171', 'Unclassified'),  # medium gray
	(8, '#aeaaaa', 'Cloud medium probability'),  # light gray
	(9, '#d0cece', 'cloud high probability'),  # very light gray
	(10, '#00ccff', 'Thin cirrus'),  # light blue/purple
	(11, '#ff66ff', 'Snow or ice'),  # cyan
]
scl_color_hex = [x[1] for x in scl_color_def]
_ = (
	# cloud_removed.isel(time=0)	# return new xarray.DataSet with each array index along the `time` dimension
	stac_assets[['scl_cloud_removed']]
	.isel(time=0)  # return new xarray.DataSet with each array index along the `time` dimension
	.to_array("band")  # convert to xarray.DataArray
	.plot.imshow(  # wraps matplotlib.pyplot.imshow()
		col="band",  # make column faceted plots on this dimension name
		size=5,  # height (in inches) of each plot.
		vmin=0,  # Lower value to anchor the colormap
		vmax=11,  # Upper value to anchor the colormap
		colors=scl_color_hex,
		levels=13,
		add_colorbar=False)
)

plt.title("Scene Classification Layer (SCL) Band - Surface Only")
plt.xlabel("longitude")
plt.ylabel("latitude")

ax = plt.gca()
ax.axis("equal")

# Create a legend object and add the patches to it
masked_color_def = [
	('X', '#404040', 'Removed'),
	(4, '#04ff00', 'Vegetation'),  # green
	(5, '#ffff00', 'Not vegetated'),  # yellow
	(6, '#0000cc', 'Water')  # blue
]
patches = []
for color in masked_color_def:
	patch = mpatches.Patch(color=color[1], label='{} - {}'.format(color[0], color[2]))
	patches.append(patch)
plt.legend(handles=patches, title='Classification', bbox_to_anchor=(1.6, 1))

# add the zone boundaries
_ = aoi_projected.plot(ax=ax, facecolor="none", edgecolor='red', lw=3)

## 2. Calculate ndvi raw

In [ ]:
stac_assets['ndvi_raw'] = XarrayUtils.calculate_ndvi(stac_assets)
print(stac_assets[['ndvi_raw']])

## 3. Fill cloud gap

In [ ]:
stac_assets["ndvi"] = XarrayUtils.fill_cloud_gap(stac_assets["scl_cloud_removed"], stac_assets['ndvi_raw'], previous_ndvi_raster)
print(stac_assets[['ndvi']])

## 4. Calculate ndvi change

In [ ]:
if previous_ndvi_raster is not None:
	stac_assets['ndvi_change'] = XarrayUtils.calculate_ndvi_change(stac_assets['ndvi'], previous_ndvi_raster)
	print(stac_assets[['ndvi_change']])

## 5. Generate metadata

In [ ]:
sentinel_link = [{"rel": "derived_from", "href": link.href, "type": link.media_type} for link in processor.stac_item.links if link.rel == 'self'].pop()

# generate metadata.json
MetadataUtils.generate_metadata(sentinel_link, processor.bounding_box.tolist(), request.coordinates, stac_assets, temp_dir, bucket_name, request.output_prefix)

# upload assets to s3
MetadataUtils.upload_assets(bucket_name, request.output_prefix, temp_dir)

## 6. Publish finish event

In [ ]:
import boto3
from datetime import datetime

# Define the event details
event_bus_name = os.getenv("OUTPUT_BUCKET")
aws_batch_job_id = os.getenv("AWS_BATCH_JOB_ID")
eventbridge = boto3.client('events')
event = {
	'EventBusName': event_bus_name,
	'Source': 'com.aws.arcade.executor',
	'DetailType': 'com.aws.arcade.executor>polygonMetadata>created',
	'Detail': json.dumps({
		"groupId": request.group_id,
		"polygonId": request.polygon_id,
		"regionId": request.region_id,
		"resultId": request.result_id,
		"jobId": aws_batch_job_id,
		"scheduleDateTime": request.schedule_date_time,
		"engineOutputLocation": f'{request.output_prefix}/metadata.json',
		"createdAt": datetime.now().isoformat()
	})}

# Publish the event to EventBridge
response = eventbridge.put_events(
	Entries=[
		{
			'Source': event['Source'],
			'DetailType': event['DetailType'],
			'Detail': event['Detail']
		}
	]
)

print(response)